In [47]:
import pandas as pd
import os

In [ ]:
os.chdir('c:/users/enowak/desktop/busstuff')

In [1]:
fn = 'c:/users/enowak/desktop/deadhead routes.xlsx'

In [13]:
df = pd.read_excel(fn, sheet_name='Routes Lat-long', index=None, header=None)

In [14]:
df = df.iloc[:, 1:]

In [45]:
d = {}

for c in df.columns:
    temp = df[c]
    name = '{0}_{1}'.format(temp[0], temp[1]).replace(' ', '')
    
    via = temp[2]
    
    temp = temp.dropna()[3:].tolist()
    
    temp_o = temp.pop(0)
    temp_d = temp.pop()
    
    if len(temp) > 0:
        temp = 'via:' + '|via:'.join(temp)
    else:
        temp = ''
    
    try:
        d[name].append([via, temp_o, temp_d, temp])
    except:
        d[name] = []


In [46]:
d['EltingvilleTC_Worth&Broadway']

[['Bayonne',
  '40.560102, -74.170633',
  '40.716054, -74.004814',
  'via:40.575410, -74.169675|via:40.599582, -74.162528|via:40.607748, -74.158556|via:40.618101, -74.151380|via:40.641176, -74.142479|via:40.673766, -74.105442|via:40.679945, -74.101418|via:40.691940, -74.083670|via:40.724736, -74.053756|via:40.727846, -74.024873|via:40.720079, -74.006853|via:40.718357, -74.005623|via:40.717156, -74.003935'],
 ['I-95',
  '40.560102, -74.170633',
  '40.716054, -74.004814',
  'via:40.533777, -74.197548|via:40.524811, -74.248045|via:40.548694, -74.278193|via:40.547601, -74.280092|via:40.652144, -74.195170|via:40.683122, -74.167404|via:40.695486, -74.119734|via:40.691940, -74.083670|via:40.724736, -74.053756|via:40.727846, -74.024873|via:40.720079, -74.006853|via:40.718357, -74.005623|via:40.717156, -74.003935'],
 ['NY-440',
  '40.560102, -74.170633',
  '40.716054, -74.004814',
  'via:40.561311, -74.171111|via:40.557983, -74.202125|via:40.628443, -74.182516|via:40.652144, -74.195170|via:40.6

In [48]:
os.chdir('c:/users/enowak/desktop/busstuff/')

In [132]:
fs = [i for i in os.listdir('.') if not i.startswith('~') and i.endswith('.xlsx')]
fs

['AnnadaleDrumgooleRdWest_WorthBroadway.xlsx',
 'CastletonJewett_PearlFrankfort.xlsx',
 'CharlestonDepot_PearlFrankfort.xlsx',
 'CharlestonDepot_WorthBroadway.xlsx',
 'CraigAveHylanBLVD_WorthBroadway.xlsx',
 'EltingvilleTC_PearlFrankfort.xlsx',
 'EltingvilleTC_TrinityRector.xlsx',
 'EltingvilleTC_WorthBroadway.xlsx',
 'MeredithDepot_PearlFrankfort.xlsx',
 'MeredithDepot_TrinityRector.xlsx',
 'MeredithDepot_WorthBroadway.xlsx',
 'RichmondTerraceSouthAv_WorthBroadway.xlsx',
 'VictoryWild_WorthBroadway.xlsx',
 'YukonDepot_PearlFrankfort.xlsx',
 'YukonDepot_WorthBroadway.xlsx']

In [81]:
f = fs[0]

In [83]:
data = pd.read_excel(f)

In [66]:
df = data.reset_index(inplace=True)

In [67]:
df.rename(columns={'index': 'date_time'}, inplace=True)

In [68]:
df.date_time = pd.to_datetime(df.date_time, infer_datetime_format=True)

In [69]:
cols = [i for i in df.columns if i.endswith('actual_time')]

In [70]:
df = df[['date_time'] + cols]

In [127]:
def get_col_name(row, cols):
    for c in cols:
        if row[c] == row['Best Travel Time']:
            return c.split('_')[0]

In [117]:
print(f)

temp = pd.DataFrame()

for key, grp in df.groupby(df.date_time.dt.hour):
    
    c = str(key) + ':00'
    
    grp_m = grp.mean().to_frame()
    grp_m.columns = [c]
    
    temp = temp.append(grp_m.T)

temp['Best Travel Time'] = temp.min(axis=1)
temp['Best Route'] = temp.apply(get_col_name, cols=cols, axis=1)

print(temp)


AnnadaleDrumgooleRdWest_WorthBroadway.xlsx
       I-95_actual_time  NY-440_actual_time  Best Travel Time  \
14:00           59.8445             59.2375           59.2375   
15:00           62.1290             61.6570           61.6570   
16:00           67.1425             66.3015           66.3015   
17:00           69.9670             68.8710           68.8710   
18:00           62.6730             61.2660           61.2660   
19:00           58.3775             56.9240           56.9240   

               Best Route  
14:00  NY-440_actual_time  
15:00  NY-440_actual_time  
16:00  NY-440_actual_time  
17:00  NY-440_actual_time  
18:00  NY-440_actual_time  
19:00  NY-440_actual_time  


In [118]:
writer = pd.ExcelWriter(f, engine='xlsxwriter')

In [119]:
data.to_excel(writer, sheet_name='Data')
temp.to_excel(writer, sheet_name='Analysis')
writer.save()
writer.close()

In [135]:
writer = pd.ExcelWriter('deadhead_analysis.xlsx', engine='xlsxwriter')

for f in fs:
    print(f[:-5])

    data = pd.read_excel(f)

    df = data.reset_index()

    df.rename(columns={'index': 'date_time'}, inplace=True)

    df.date_time = pd.to_datetime(df.date_time, infer_datetime_format=True)

    cols = [i for i in df.columns if i.endswith('actual_time')]

    df = df[['date_time'] + cols]

    temp = pd.DataFrame()

    for key, grp in df.groupby(df.date_time.dt.hour):

        c = str(key) + ':00'

        grp_m = grp.mean().to_frame()
        grp_m.columns = [c]

        temp = temp.append(grp_m.T)

    temp['Best Travel Time'] = temp.min(axis=1)
    temp['Best Route'] = temp.apply(get_col_name, cols=cols, axis=1)
    
    temp['Saved Travel Time'] = temp['HC Tunnel_actual_time'] - temp['Best Travel Time']
    
    new_cols = [i.split('_')[0] for i in cols]
    
    temp.rename(columns=dict(zip(cols, new_cols)), inplace=True)
    
    temp = temp[['Best Route', 'Best Travel Time', 'Saved Travel Time'] + new_cols]
    
    temp.index.name = 'Time'

    temp.to_excel(writer, sheet_name=f[:-5][:31])
writer.save()
writer.close()
    
#     writer = pd.ExcelWriter(f, engine='xlsxwriter')

#     data.to_excel(writer, sheet_name='Data')
#     temp.to_excel(writer, sheet_name='Analysis')
    
#     writer.save()
#     writer.close()

AnnadaleDrumgooleRdWest_WorthBroadway
CastletonJewett_PearlFrankfort
CharlestonDepot_PearlFrankfort
CharlestonDepot_WorthBroadway
CraigAveHylanBLVD_WorthBroadway
EltingvilleTC_PearlFrankfort
EltingvilleTC_TrinityRector
EltingvilleTC_WorthBroadway
MeredithDepot_PearlFrankfort
MeredithDepot_TrinityRector
MeredithDepot_WorthBroadway
RichmondTerraceSouthAv_WorthBroadway
VictoryWild_WorthBroadway
YukonDepot_PearlFrankfort
YukonDepot_WorthBroadway


In [154]:
writer = pd.ExcelWriter('deadhead_analysis.xlsx', engine='xlsxwriter')

for f in fs:
    print(f[:-5])

    data = pd.read_excel('si_to_ma/' + f)

    df = data.reset_index()

    df.rename(columns={'index': 'date_time'}, inplace=True)

    df.date_time = pd.to_datetime(df.date_time, infer_datetime_format=True)

    cols = [i for i in df.columns if i.endswith('actual_time')]

    df = df[['date_time'] + cols]
    
    df = df[(df.date_time.dt.hour >= 16) & (df.date_time.dt.hour < 20)]
    
#     temp = pd.DataFrame()

#     for key, grp in df.groupby(df.date_time.dt.hour):

#         c = str(key) + ':00'

#         grp_m = grp.std().to_frame()
#         grp_m.columns = [c]
        
#         temp = temp.append(grp_m.T)
    
    temp = df.describe()
    
#     print(temp)
#     raise Exception('lkasjdf')
    
    temp['Best Travel Time'] = temp.min(axis=1)
    temp['Best Route'] = temp.apply(get_col_name, cols=cols, axis=1)
    
    temp['Saved Travel Time'] = temp['HC Tunnel_actual_time'] - temp['Best Travel Time']
    
    new_cols = [i.split('_')[0] for i in cols]
    
    temp.rename(columns=dict(zip(cols, new_cols)), inplace=True)
    
    temp = temp[new_cols]
    
    temp.index.name = 'Time'

    temp.to_excel(writer, sheet_name=f[:-5][:31])
writer.save()
writer.close()

AnnadaleDrumgooleRdWest_WorthBroadway
CastletonJewett_PearlFrankfort
CharlestonDepot_PearlFrankfort
CharlestonDepot_WorthBroadway
CraigAveHylanBLVD_WorthBroadway
EltingvilleTC_PearlFrankfort
EltingvilleTC_TrinityRector
EltingvilleTC_WorthBroadway
MeredithDepot_PearlFrankfort
MeredithDepot_TrinityRector
MeredithDepot_WorthBroadway
RichmondTerraceSouthAv_WorthBroadway
VictoryWild_WorthBroadway
YukonDepot_PearlFrankfort
YukonDepot_WorthBroadway


In [131]:
for f in fs:
    print(f)
    
    df1 = pd.read_excel(f)
    df2 = pd.read_excel('x/' + f)
    
    df12 = pd.concat([df1, df2], axis=1)
    
    df12.to_excel(f)

AnnadaleDrumgooleRdWest_WorthBroadway.xlsx
CastletonJewett_PearlFrankfort.xlsx
CharlestonDepot_PearlFrankfort.xlsx
CharlestonDepot_WorthBroadway.xlsx
CraigAveHylanBLVD_WorthBroadway.xlsx
EltingvilleTC_PearlFrankfort.xlsx
EltingvilleTC_TrinityRector.xlsx
EltingvilleTC_WorthBroadway.xlsx
MeredithDepot_PearlFrankfort.xlsx
MeredithDepot_TrinityRector.xlsx
MeredithDepot_WorthBroadway.xlsx
RichmondTerraceSouthAv_WorthBroadway.xlsx
VictoryWild_WorthBroadway.xlsx
YukonDepot_PearlFrankfort.xlsx
YukonDepot_WorthBroadway.xlsx
